In [1]:
with open('EVT_2023_05_10_13_19_27_F_gps.txt', encoding='utf-8') as f:
    texts = f.read().split('\n')

In [2]:
import numpy as np
import pandas as pd

###
columns_name = ["index","time","text","info"]
###

raw_data = []
data = []
for t in texts:
    if(t == ""):
        data.append(raw_data)
        raw_data = []
    else:
        raw_data.append(t)

data = pd.DataFrame(data,columns=columns_name)
data = data.set_index("index")
data

,time,text,info
index,,,
1,"00:00:00,000 --> 00:00:00,100","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
2,"00:00:00,100 --> 00:00:00,200","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
3,"00:00:00,200 --> 00:00:00,300","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
4,"00:00:00,300 --> 00:00:00,400","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
5,"00:00:00,400 --> 00:00:00,500","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
...,...,...,...
197,"00:00:19,600 --> 00:00:19,700","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None
198,"00:00:19,700 --> 00:00:19,800","<font size=""0"" color=""#000000"">{\an7}gsensori,...",";CAR,0,0,0,0.0,0,0,0,0,0,0,0,0</font>"
199,"00:00:19,800 --> 00:00:19,900","<font size=""0"" color=""#000000"">{\an7}gsensori,...",None


In [3]:
clean_data = data.drop(["info"],axis=1)

texts : list[str] = clean_data["text"].to_list()


acceleration_x,acceleration_y,acceleration_z = [],[],[]
# 加速度
for text in texts:
    text = str(text)
    index = text.find("gsensori")
    acceleration = text[index+9+6:index+9+6+11].split(",")
    if(len(acceleration) != 3):
        continue
    acceleration_x.append(acceleration[0])
    acceleration_y.append(acceleration[1])
    acceleration_z.append(acceleration[2])

# 經緯度
#! 文字数で指定しているので緯度経度が大きく変化するとエラーが出る
latitude,latitude_str,longitude,longitude_str = [],[],[],[]
for text in texts:
    text = str(text)
    index = text.find("GPRMC")
    if(index == -1):
        continue
    latitude.append(text[index+5+13:index+5+13+11])
    latitude_str.append(text[index+5+13+11+1:index+5+13+11+1+1])
    longitude.append(text[index+5+13+11+2+1:index+5+13+11+1+2+12])
    longitude_str.append(text[index+5+13+11+2+1+12+1:index+5+13+11+1+2+12+1+1])

latitude = [float(e) for e in latitude]
latitude_str = [str(e) for e in latitude_str]
longitude = [float(w) for w in longitude]
longitude_str = [str(w) for w in longitude_str]

position = pd.DataFrame([latitude,latitude_str,longitude,longitude_str]).T
position = position.rename(columns={0:"latitude",1:"latitude_str",2:"longitude",3:"longitude_str"})

In [5]:
position

,latitude,latitude_str,longitude,longitude_str
0,3703.867911,N,14052.997824,E
1,3703.862235,N,14052.999685,E
2,3703.856208,N,14053.001337,E
3,3703.856208,N,14053.001337,E
4,3703.844659,N,14053.005956,E
5,3703.839392,N,14053.008985,E
6,3703.834919,N,14053.013599,E
7,3703.83054,N,14053.018267,E
8,3703.825342,N,14053.023067,E
9,3703.821114,N,14053.028543,E
